In [1]:
import time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import pyplot
import matplotlib.cm as cm
import matplotlib.colors as norm
from matplotlib.gridspec import SubplotSpec
import seaborn as sns

from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline #This allows one to build different steps together



In [2]:
def log_mass_size(df):
    '''
    Converts to log10 non-zero size and mass entries (without distinguishing galaxy type)
    output: df with new columns of log mass, log size added to it

    '''
    #first let's remove all galaxy_mass=0 since you are going to at the end anyway
    df=df[df['mstar'] > 0]

#     #subhalo_flag = subhalos[‘SubhaloFlag’]
#     if type_of_galaxy == 'central':
#         df=df[df['subhalo_flag']==True]
#     elif type_of_galaxy == 'satellite':
#         df=df[df['subhalo_flag']==False]
#     else: 
#         print("type_of_galaxy must be set to central or satellite")

    #since we got rid of all zero mass we can just take the log10
    df.loc[:, 'mstar_log']=np.log10(df.loc[:, 'mstar'])
    df.loc[:, 'half_radius_log']=np.log10(df.loc[:, 'half_radius'])
    df.loc[:, 'mhalo_log']=np.log10(df.loc[:, 'mhalo'])
    df.loc[:, 'Mhot_log']=np.log10(df.loc[:, 'Mhot'])
    df.loc[:,'mbulge_log']=df.loc[:,'mbulge'].apply(lambda x: np.log10(x) if x>0 else 0) # because there are 0 values of mbulge
    df.loc[:,'SFR_log']=df.loc[:,'sfr'].apply(lambda x: np.log10(x) if x>0 else 0)


    return df

In [3]:
df=pd.read_hdf('./tng-sam_v2.h5') 


In [4]:
df

,mhalo,spin,Cnfw,Mhot,Macc,mstar,mHI,mH2,mbulge,r_bulge,r_disk,sfr,Zcold,Zstar,sat_type
0,8.945970e+08,0.03410,27.518700,1.150910e+08,0.0,1.918680e+04,1443460.0,350.895996,1.705140e+03,1.380600e-07,0.612071,2.099810e-07,6.425340e-06,1.847320e-08,0.0
1,1.815770e+09,0.04884,25.676800,9.966590e+07,0.0,5.311550e+05,1239300.0,574.640991,9.893530e+04,3.582890e-04,1.109940,3.485530e-07,5.455150e-05,2.259670e-06,0.0
2,4.880430e+09,0.07510,24.664200,1.409960e+07,0.0,1.461610e+06,51356600.0,120703.000000,2.745250e+05,3.465510e-03,1.129830,7.226970e-05,6.013470e-04,7.569950e-06,0.0
3,1.860050e+09,0.03953,19.896200,1.573440e+08,0.0,5.375590e+03,3270910.0,381.580017,2.401990e+03,5.426130e-08,0.905608,2.283380e-07,5.815860e-06,2.387450e-09,0.0
4,1.615000e+09,0.04429,28.457001,1.273490e+08,0.0,1.164560e+04,655190.0,66.861099,0.000000e+00,0.000000e+00,0.967984,4.000970e-08,6.864240e-06,1.009730e-08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924830,8.857400e+08,0.01890,24.130699,9.038510e+07,0.0,1.155000e+03,274128.0,58.854900,4.672700e+01,1.623750e-10,0.357797,3.521890e-08,9.629620e-07,3.182000e-10,0.0
1924831,9.477410e+08,0.04871,18.530899,9.490910e+07,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.000000e+00,0.891291,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1924832,1.617950e+09,0.03615,34.817600,1.473450e+08,0.0,6.519520e+03,589709.0,66.945503,1.819190e+03,1.039700e-07,0.790560,4.069460e-08,5.339940e-06,3.317030e-09,0.0
1924833,2.970180e+09,0.01930,15.964400,1.480060e+08,0.0,6.943700e+04,15834101.0,7420.919922,4.825620e+03,4.585310e-07,0.535545,4.442270e-06,3.334850e-05,9.543090e-08,0.0


In [5]:
df.shape

(1924835, 15)

In [6]:
df['sat_type'].value_counts() # sat_type=0 corresponds to CENTRAL galaxies; 1,346,899 central galaxies

0.0    1346899
1.0     475176
2.0      91982
3.0      10122
4.0        611
5.0         45
Name: sat_type, dtype: int64

### <font color='blue'> Filter the dataset

#### Mass filter


In [7]:
# remove all galaxies with stellar mass < 1e8
df_mass_filtered = df.loc[df.mstar>1e8,:]

In [8]:
df_mass_filtered.shape # 51,906 galaxies with stellar mass >1e8

(51906, 15)

#### Type filter (choose centrals only)

In [9]:
df_mass_filtered['sat_type'].value_counts() # 35,390 central galaxies with stellar mass >1e8

0.0    35390
1.0    13686
2.0     2540
3.0      271
4.0       18
5.0        1
Name: sat_type, dtype: int64

In [10]:
df_central=df_mass_filtered.loc[df_mass_filtered.sat_type==0,:] # choose only CENTRALs

In [11]:
df_central

,mhalo,spin,Cnfw,Mhot,Macc,mstar,mHI,mH2,mbulge,r_bulge,r_disk,sfr,Zcold,Zstar,sat_type
53,4.502510e+09,0.03378,59.129101,9.503960e+09,0.0,1.050790e+08,2.710510e-11,0.0,72604704.0,0.503721,1.03908,0.000402,0.000008,0.004245,0.0
59,1.285310e+11,0.03407,21.878799,8.516240e+05,0.0,2.136860e+09,4.310100e+08,145516000.0,231256000.0,1.340100,1.44021,0.101277,0.206532,0.485612,0.0
82,1.171930e+11,0.01407,23.447201,5.588210e+05,0.0,1.583780e+09,3.059520e+08,46185400.0,240634992.0,1.801320,1.82321,0.028763,0.099900,0.308089,0.0
98,1.705850e+11,0.02229,17.701000,1.107320e+06,0.0,1.241640e+09,5.498750e+08,68248200.0,323452000.0,2.040110,2.30283,0.042378,0.160787,0.204719,0.0
104,8.294760e+10,0.01128,42.640499,1.935800e+05,0.0,1.432650e+09,1.699390e+08,24481200.0,490328000.0,1.414440,1.62481,0.015081,0.061155,0.310911,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918905,5.192210e+10,0.03800,24.981800,3.574300e+04,0.0,1.312080e+08,2.422220e+08,7387440.0,7870290.5,0.861413,2.64082,0.004443,0.028542,0.007200,0.0
1918910,5.454680e+10,0.04917,16.239500,4.573560e+08,0.0,1.234320e+08,2.204080e+08,4975330.0,10646200.0,1.976380,3.47372,0.002984,0.036281,0.005770,0.0
1918945,5.175670e+10,0.03464,20.015400,1.392480e+08,0.0,1.227350e+08,4.174370e+08,75087504.0,20440200.0,0.553882,1.15671,0.051229,0.026656,0.006112,0.0
1918947,5.431060e+10,0.04613,20.577600,2.903790e+07,0.0,1.518140e+08,4.901590e+08,39315300.0,17960600.0,0.168341,1.62632,0.024660,0.041628,0.009348,0.0


In [12]:
df_central['sat_type'].value_counts()

0.0    35390
Name: sat_type, dtype: int64

In [13]:
np.log10(df_central['mstar'].max())

11.960914

In [14]:
np.log10(df_central['mhalo'].max())

14.686748

### <font color='blue'> Add half_radius by applying half_mass_radius function to each row


In [15]:
# import numpy as np
import matplotlib.pyplot as plt
import galsim #install with conda install -c conda_forge galsim

def half_mass_radius(mstar,mbulge,r_disk,r_bulge,tol=1.e-6,figure=False):
    '''calculate half mass radius for disk(n=1)+bulge(n=4) galaxy
    using bisection'''
    
    Md=mstar-mbulge # disk mass=stellar mass - bulge mass
    Rd=r_disk # disk radius
    Mb=mbulge # bulge mass
    Rb=r_bulge # bulge radius
    
    disk_fraction=Md/(Md+Mb)
    bulge_fraction=Mb/(Md+Mb)
    if bulge_fraction==0:
        disk=galsim.Sersic(scale_radius=Rd,n=1)  
        return disk.half_light_radius     
    if disk_fraction==0:
        return Rb
    disk=galsim.Sersic(scale_radius=Rd,n=1)
    bulge=galsim.Sersic(half_light_radius=Rb,n=4)
    #starting points for bisection
    a=bulge.half_light_radius
    b=disk.half_light_radius
    if b < a:
        a=disk.half_light_radius
        b=bulge.half_light_radius

    #bisection
    Ma=disk_fraction*disk.calculateIntegratedFlux(a)+bulge_fraction*bulge.calculateIntegratedFlux(a)
    Mb=disk_fraction*disk.calculateIntegratedFlux(b)+bulge_fraction*bulge.calculateIntegratedFlux(b)
    if np.sign(Ma-0.5)==np.sign(Mb-0.5):
        raise Exception("a and b do not bound a root")
    while(b-a > tol):
        m=0.5*(a+b)
        f=(disk_fraction*disk.calculateIntegratedFlux(m)+
            bulge_fraction*bulge.calculateIntegratedFlux(m))-0.5
        if np.sign(f)==1:
            b=m
        else:
            a=m
    half_radius=0.5*(a+b)
    if figure:
        half=np.array([0.45,0.55]) #used for half marks
        R=np.arange(0,3*b,0.02)
        N=len(R)
        yd=np.zeros(N)
        yb=np.zeros(N)
        for i in range(N):
            yd[i]=disk_fraction*disk.calculateIntegratedFlux(R[i])
            yb[i]=bulge_fraction*bulge.calculateIntegratedFlux(R[i])
        plt.plot(R,yd,color='b',label='disk')
        plt.plot(R,yb,color='r',label='bulge')
        plt.plot(R,yd+yb,color='k',label='total')
        plt.plot([bulge.half_light_radius,bulge.half_light_radius],bulge_fraction*half,color='g')
        plt.plot([disk.half_light_radius,disk.half_light_radius],disk_fraction*half,color='g')
        plt.plot([half_radius,half_radius],half,color='g')
        plt.xlim([0,3*b])
        plt.ylim([0,1.0])
        plt.legend()
        plt.show()
    return half_radius

# if __name__=='__main__':
#     print(half_mass_radius(1.e10,3.5,0.e9,0.0,figure=True))

In [17]:
df_central.iloc[101,:]

mhalo       6.778860e+10
spin        2.908000e-02
Cnfw        8.621200e+00
Mhot        2.139400e+07
Macc        0.000000e+00
mstar       1.449540e+09
mHI         0.000000e+00
mH2         0.000000e+00
mbulge      1.439710e+09
r_bulge     3.956790e-01
r_disk      2.208780e+00
sfr         6.302250e-02
Zcold       1.938820e-03
Zstar       1.716930e+00
sat_type    0.000000e+00
Name: 2093, dtype: float32

In [18]:
#for i,row in df_central.iterrows():
df_central.loc[:,'half_radius'] = df_central.apply(lambda x: half_mass_radius(mstar=x['mstar'],mbulge=x['mbulge'],r_disk=x['r_disk'],r_bulge=x['r_bulge'],tol=1.e-6,figure=False), axis=1)

/Users/festabu/miniforge3/envs/native/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/festabu/miniforge3/envs/native/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [19]:
df_central.shape

(35390, 16)

In [20]:
df_central['half_radius']

53         0.829693
59         2.332662
82         2.922134
98         3.491710
104        2.360157
             ...   
1918905    4.229993
1918910    5.548773
1918945    1.729907
1918947    2.406906
1918957    2.117134
Name: half_radius, Length: 35390, dtype: float64

In [21]:
# solution of the same problem using a for loop over the df rows
lst = []
for i,row in df_central.iterrows():
    result_iter = half_mass_radius(mstar=row['mstar'],mbulge=row['mbulge'],r_disk=row['r_disk'],r_bulge=row['r_bulge'],tol=1.e-6,figure=False)
    lst.append(result_iter)
    if i>100:
        break
               

In [22]:
lst

[0.8296922716638879,
 2.3326619620962354,
 2.922134029718011,
 3.4917102387445444,
 2.3601561747952067]

### <font color='blue'> Logarithmic scale of stellar mass, SFR, Mhalo

In [24]:
df_central.head(2)

,mhalo,spin,Cnfw,Mhot,Macc,mstar,mHI,mH2,mbulge,r_bulge,r_disk,sfr,Zcold,Zstar,sat_type,half_radius
53,4.502510e+09,0.03378,59.129101,9.503960e+09,0.0,1.050790e+08,2.710510e-11,0.0,72604704.0,0.503721,1.03908,0.000402,0.000008,0.004245,0.0,0.829693
59,1.285310e+11,0.03407,21.878799,8.516240e+05,0.0,2.136860e+09,4.310100e+08,145516000.0,231256000.0,1.340100,1.44021,0.101277,0.206532,0.485612,0.0,2.332662


In [25]:
df_central_log=log_mass_size(df_central)

In [28]:
df_central_log.head(2)

,mhalo,spin,Cnfw,Mhot,Macc,mstar,mHI,mH2,mbulge,r_bulge,...,Zcold,Zstar,sat_type,half_radius,mstar_log,half_radius_log,mhalo_log,Mhot_log,mbulge_log,SFR_log
53,4.502510e+09,0.03378,59.129101,9.503960e+09,0.0,1.050790e+08,2.710510e-11,0.0,72604704.0,0.503721,...,0.000008,0.004245,0.0,0.829693,8.021516,-0.081083,9.653455,9.977904,7.860965,-3.395986
59,1.285310e+11,0.03407,21.878799,8.516240e+05,0.0,2.136860e+09,4.310100e+08,145516000.0,231256000.0,1.340100,...,0.206532,0.485612,0.0,2.332662,9.329776,0.367852,11.109008,5.930248,8.364093,-0.994489


In [29]:
df_central_log['mbulge_log'].max()

11.92681558852368

In [30]:
df_central_log['mbulge_log'].min()

0.0

### Gas fraction


In [31]:
df_central_log['neutral_H_mass']=(df_central_log['mHI']+df_central_log['mH2'])
df_central_log['baryon_mass']=df_central_log['neutral_H_mass']+df_central_log['mstar']
df_central_log['gas_fraction']=df_central_log['neutral_H_mass']/df_central_log['baryon_mass']

In [32]:
df_central_log.shape

(35390, 25)

In [33]:
np.count_nonzero(df_central_log['half_radius']) # this means that there are no size zero entries in TNG-SAM

35390

In [34]:
np.count_nonzero(df_central_log['mstar']) # this means that there are no stellar mass zero entries in TNG-SAM

35390

In [35]:
np.count_nonzero(df_central_log['mhalo'])

35390

In [36]:
df_central_log.SFR_log.max()

2.0902862818924777

In [37]:
df_central_log.SFR_log.min()

-9.442787629874614

### Normalize the dataset (dimensionless)

In [38]:
df_central_log.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['mhalo', 'spin', 'Cnfw', 'Mhot', 'Macc', 'mstar', 'mHI', 'mH2',
       'mbulge', 'r_bulge', 'r_disk', 'sfr', 'Zcold', 'Zstar', 'sat_type',
       'half_radius', 'mstar_log', 'half_radius_log', 'mhalo_log', 'Mhot_log',
       'mbulge_log', 'SFR_log', 'neutral_H_mass', 'baryon_mass',
       'gas_fraction'],
      dtype='object')>

In [39]:
def normalization_func(df):
    '''
    Normalizes the dataset by dividing all masses by halo mass, and galaxy size by halo size

    '''
#     could also use the method below
#     df_log_mass_filtered.loc[:,'GalpropNormMstar']=df_log_mass_filtered.loc[:,'GalpropMstar'].div(df_log_mass_filtered.HalopropMvir, axis=0)
    
    halomass=df.loc[:,'mhalo']
#     halorad= df.loc[:,'GalpropRhalo']
    
#     df.loc[:,'GalpropNormHalfRadius']=df.loc[:,'GalpropHalfRadius']/halorad
#     df.loc[:,'GalpropNormRhalo']=df.loc[:,'GalpropRhalo']/halorad

    df.loc[:,'GalpropNormMstar']=df.loc[:,'mstar']/halomass
#     df.loc[:,'GalpropNormMvir']=df.loc[:,'GalpropMvir']/halomass
    df.loc[:,'HalopropNormMhot']=df.loc[:,'Mhot']/halomass
    df.loc[:,'GalpropNormMbulge']=df.loc[:,'mbulge']/halomass
    
#     df.loc[:,'GalpropNormMBH']=df.loc[:,'GalpropMBH']/halomass
    df.loc[:,'GalpropNormMH2']=df.loc[:,'mH2']/halomass
    df.loc[:,'GalpropNormMHI']=df.loc[:,'mHI']/halomass
#     df.loc[:,'GalpropNormMHII']=df.loc[:,'GalpropMHII']/halomass
#     df.loc[:,'GalpropNormMcold']=df.loc[:,'GalpropMcold']/halomass
#     df.loc[:,'GalpropNormMstar_merge']=df.loc[:,'GalpropMstar_merge']/halomass
#     df.loc[:,'GalpropNormMstrip']=df.loc[:,'GalpropMstrip']/halomass
    
    return df

In [40]:
df_central_normalized=normalization_func(df_central_log)

In [42]:
df_central_normalized.shape

(35390, 30)

In [44]:
df_central_normalized.head(2)

,mhalo,spin,Cnfw,Mhot,Macc,mstar,mHI,mH2,mbulge,r_bulge,...,mbulge_log,SFR_log,neutral_H_mass,baryon_mass,gas_fraction,GalpropNormMstar,HalopropNormMhot,GalpropNormMbulge,GalpropNormMH2,GalpropNormMHI
53,4.502510e+09,0.03378,59.129101,9.503960e+09,0.0,1.050790e+08,2.710510e-11,0.0,72604704.0,0.503721,...,7.860965,-3.395986,2.710510e-11,1.050790e+08,2.579497e-19,0.023338,2.110814,0.016125,0.000000,6.019998e-21
59,1.285310e+11,0.03407,21.878799,8.516240e+05,0.0,2.136860e+09,4.310100e+08,145516000.0,231256000.0,1.340100,...,8.364093,-0.994489,5.765260e+08,2.713386e+09,2.124747e-01,0.016625,0.000007,0.001799,0.001132,3.353355e-03


### The final dataset

In [46]:
df_central_log.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['mhalo', 'spin', 'Cnfw', 'Mhot', 'Macc', 'mstar', 'mHI', 'mH2',
       'mbulge', 'r_bulge', 'r_disk', 'sfr', 'Zcold', 'Zstar', 'sat_type',
       'half_radius', 'mstar_log', 'half_radius_log', 'mhalo_log', 'Mhot_log',
       'mbulge_log', 'SFR_log', 'neutral_H_mass', 'baryon_mass',
       'gas_fraction', 'GalpropNormMstar', 'HalopropNormMhot',
       'GalpropNormMbulge', 'GalpropNormMH2', 'GalpropNormMHI'],
      dtype='object')>

In [47]:
df_central_log_pairplot = df_central_log.loc[:,["half_radius", 'spin', 'sfr', 
                                                "Cnfw", 'Zcold', 'Zstar',
                                                'GalpropNormMstar', 'HalopropNormMhot',
                                                'GalpropNormMbulge', 'GalpropNormMH2', 'GalpropNormMHI']]

In [48]:
df_central_log_pairplot.shape

(35390, 11)

In [49]:
# df_central_log_pairplot.to_csv('df_central_log_pairplot_10feat_v3.csv', index=False)

In [52]:
df_central_log_pairplot = pd.read_csv ('df_central_log_pairplot_10feat_v3.csv') # df here read from the csv in previous line

In [53]:
df_central_log_pairplot.head(2)

,half_radius,spin,sfr,Cnfw,Zcold,Zstar,GalpropNormMstar,HalopropNormMhot,GalpropNormMbulge,GalpropNormMH2,GalpropNormMHI
0,0.829693,0.03378,0.000402,59.1291,0.000008,0.004245,0.023338,2.110814,0.016125,0.000000,6.019997e-21
1,2.332662,0.03407,0.101277,21.8788,0.206532,0.485612,0.016625,0.000007,0.001799,0.001132,3.353354e-03
